### 01. Coco Descriptions

This notebook generates `coco_descriptions.pkl`

In [37]:
import re
import time
import pickle

from pycocotools.coco import COCO

In [ ]:
def decontracted(phrase):
    # specific
    phrase = re.sub(r"won't", "will not", phrase)
    phrase = re.sub(r"can\'t", "can not", phrase)

    # general
    phrase = re.sub(r"n\'t", " not", phrase)
    phrase = re.sub(r"\'re", " are", phrase)
    phrase = re.sub(r"\'s", " is", phrase)
    phrase = re.sub(r"\'d", " would", phrase)
    phrase = re.sub(r"\'ll", " will", phrase)
    phrase = re.sub(r"\'t", " not", phrase)
    phrase = re.sub(r"\'ve", " have", phrase)
    phrase = re.sub(r"\'m", " am", phrase)
    return phrase

In [ ]:
def add_suffixes_and_prefixes(descriptions):
    for k in descriptions.keys():
        value = descriptions[k]
        caption_list = []
        for ec in value:

            # replaces specific and general phrases
            sent = decontracted(ec)
            sent = sent.replace('\\r', ' ')
            sent = sent.replace('\\"', ' ')
            sent = sent.replace('\\n', ' ')
            sent = re.sub('[^A-Za-z0-9]+', ' ', sent)

            # startseq is for kick starting the partial sequence generation and endseq is to stop while predicting.
            # for more referance please check https://machinelearningmastery.com/develop-a-deep-learning-caption-generation-model-in-python/
            image_cap = 'startseq ' + sent.lower() + ' endseq'
            caption_list.append(image_cap)
        descriptions[k] = caption_list
    return descriptions

In [ ]:
dataDir='coco'
dataType='train2014'
annFile='{}/annotations/captions_{}.json'.format(dataDir,dataType)

In [ ]:
coco=COCO(annFile)

In [ ]:
annIds = coco.getAnnIds(imgIds=35783)
anns = coco.loadAnns(annIds)
coco.showAnns(anns)
# print(type(coco.getImgIds()[0]))

In [ ]:
descriptions = {}
imgIds = coco.getImgIds()
# imgIds = [151, 260, 307, 404, 450, 491, 514, 529, 575, 671] # dummy list because I don't have all images extracted

# print(len(imgIds))
start = time.time()
for imgId in imgIds:
    annIds = coco.getAnnIds(imgIds=imgId)
    # print(len(annIds))
    anns = coco.loadAnns(annIds)
    for annotation in anns:
        if imgId in descriptions:
            descriptions[imgId].append(annotation['caption'])
        else:
            descriptions[imgId] = list()
            descriptions[imgId].append(annotation['caption'])
print("Created Descriptions Dict in {:0.2f}s".format(time.time() - start))

In [ ]:
start = time.time()
descriptions = add_suffixes_and_prefixes(descriptions)
print("Added suffixes and prefixes in {:0.2f}s".format(time.time() - start))


In [ ]:
for k, v in descriptions.items():
    print(v)
    break

In [ ]:
len(descriptions)

In [3]:
def dump_descriptions(descriptions):
    """Dump processed captions into a pickle"""
    with open("coco_descriptions.pkl", "wb") as f:
        pickle.dump(descriptions, f)

def load_descriptions(file_path):
    with open(file_path, "rb") as f:
        return pickle.load(f)

In [ ]:
dump_descriptions(descriptions)